# PyA new getitem and setitem (with modes)

In [ ]:
import numpy as np
from pya import *
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(level=logging.WARNING)

In [ ]:
plt.rcParams['figure.figsize'] = [8.0, 1]
pya.startup()

In [ ]:
# create some test signals
dur = 3.5
sr = 1000
ts = np.linspace(0, dur, int(dur*sr))
sig = np.sin(2*np.pi*50*ts**1.9)
a1 = Asig(sig, sr=1000, channels=1, cn=['a'], label='1ch-sig')
ak = Asig(np.tile(sig.reshape(3500,1), (1,4)), sr=1000, 
          label='4ch-sig', cn=['a', 'b', 'c', 'd'])
print("", a1,"\n", ak)

# getitem - row indexing

In [ ]:
a1[5] # row int

In [ ]:
a1[1000:2000:5].plot() # row slice

In [ ]:
a1[2000:1000:-5].plot() # row slice

In [ ]:
a1[[3,4,5,6,7,8,12]].plot(marker="o", lw=0) # row list

In [ ]:
a1[np.where(sig>0.8)].plot(lw=1) # row list of booleans

In [ ]:
a1[{0.5:1}].plot(lw=1) # row time slice

# getitem - column indexing

In [ ]:
print(a1[:, 0]) # col int
ak[:, 2].plot() # col int

In [ ]:
ak[:,2:4].plot(offset=4) # column slice

In [ ]:
ak[:,[0,1,3]].plot(offset=4) # column list 

In [ ]:
ak[:,['a','c']].plot(offset=4) # column by names in list

In [ ]:
ak[:,'c'].plot(offset=4) # column by single name

In [ ]:
ak[:,[True, True, False, True]].plot(offset=4) # column by bool list

# getitem - row & column indexing

In [ ]:
ak[1000:3000:10,[0,1,3]].plot(offset=4) # column list 

* seems to work, no extensive test done, but should be fine...

In [ ]:
ak2 = Asig(5.0, channels=4, sr=1000, label='canvas', cn=['1', '2', '3', '4'])
ak2

# setitem and getitem - mode None = numpy default

**mode None = numpy default conventions for array assignments**

**basic test with lists and consts**

In [ ]:
a = Asig(60, sr=10, channels=4)
b = a[:10,0]

In [ ]:
# test with ndim==1 signal
b[3:6] = [1,2,3]
print(b.sig)
b[3:6] = np.array([3,4,5])
print(b.sig)

In [ ]:
# test with ndim=2 signal
a.sig[50:55,[0,1]] = [7, 4]
a[49:56].sig

In [ ]:
a[50:55,[0,1]] = np.array([1,2])
a.sig[50:]

In [ ]:
a[50:90,0] = 4
a.plot(offset=1)

**now some tests with multichannel audio**

In [ ]:
az = Asig(5.0, sr=1000, channels=4, cn=['c1','c2','c3','c4']).plot(offset=1)

In [ ]:
az[:2000,['c1']] = -1*ak[:2000,['b']]  # invert phase of first 2 secs of ch 'a'
az.plot(offset=2)

In [ ]:
az[1000:4500,1] = ak[::-1,2].fade_in(0.2).fade_out(0.5)
az.plot(offset=2)

In [ ]:
az[:3500,2] = a1
az.plot(offset=2)

In [ ]:
az[1000:2000,3] = a1[{0:1},:]
az.plot(offset=2)

In [ ]:
az[{4:None},3] = ak[{2:3},3].fade_out(0.3)
az.plot(offset=2)

# setitem mode 'bound': truncate src at end of destination

In [ ]:
a1.b[:] += 0.4*ak[-500:,3]; a1.plot()

In [ ]:
a1.b[{3.3:None}] = 2*ak[-400:,['d']]
a1.plot()

In [ ]:
a1.b[{3.3:None},0] = -8*ak[-600:,2:3]; a1.plot()

In [ ]:
# a1.b[{3.3:None},0] = -8*ak[-600:,0]; a1.plot()

In [ ]:
a1 = pya.sine(freq=100, dur=0.2, sr=1000).fade_out(0.1).plot()

# bound add test

In [ ]:
# use time slice to mix faded sine tones to a canvas ac
ac = Asig(4.0, sr=8000)
for i in range(200):
    ac.b[{0.02*i:None}] += 0.5*pya.sine(freq=400+10*i, dur=0.2, sr=8000).fade_in(0.01).fade_out(0.15)
ac.plot().play(onset=0.3)

In [ ]:
a = Asig(1.0, sr=100, channels=2, cn=['a','b'])

In [ ]:
a.b[90:100,:] = 4.0  # set both channels
a.plot(offset=2)

In [ ]:
a[50:60,0] = 2.0
a.plot(offset=5)

In [ ]:
a.sig[:,[0,1]] *= [[1,0.3]]
a.plot(offset=5)

In [ ]:
a[:,[0,1]] *= [1,2]
a.plot(offset=2)

In [ ]:
a = pya.sine(freq=400, dur=0.2, sr=8000)
a[{0.05:0.1}] = 0
a.b[{0.075:0.077}] = 4
a.plot()

In [ ]:
a1.sig[100:150] = 0.5
a1.plot()

# setmode extend - first working example :-)

In [ ]:
a = Asig(0.8, sr=1000, channels=4, cn=['a', 'b', 'c', 'd'])
b = pya.sine(dur=0.6, freq=100, sr=1000).fade_in(0.3).fade_out(0.3)

In [ ]:
b.plot()

In [ ]:
# test with extend set mono signal to a, initially only 0.8secs long...
a.x[:,0] = 0.2*b  # this fits in without need to extend
print(a)
a.x[300:,1] = 0.5*b   
print(a)
a.x[1300:,'c'] = 0.2*b[::2] # compressed sig in ch 'c'
print(a)
a.x[1900:,3] = 0.2*b[300:] # only end of tone in ch 'd'
print(a)
a.plot(offset=1)

**extend has still lots of issues - e.g. shape problems, += doesn't work etc.**